In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, BatchNormalization

input_shape = (227, 227, 3)

model = Sequential()

model.add(Conv2D(filters=96, kernel_size=(11, 11), strides=(4, 4), activation='relu', input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))
model.add(BatchNormalization())

model.add(Conv2D(filters=256, kernel_size=(5, 5), strides=(1, 1), activation='relu', padding="same"))
model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))
model.add(BatchNormalization())

model.add(Conv2D(filters=384, kernel_size=(3, 3), strides=(1, 1), activation='relu', padding="same"))
model.add(BatchNormalization())

model.add(Conv2D(filters=384, kernel_size=(3, 3), strides=(1, 1), activation='relu', padding="same"))
model.add(BatchNormalization())

model.add(Conv2D(filters=256, kernel_size=(3, 3), strides=(1, 1), activation='relu', padding="same"))
model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))
model.add(BatchNormalization())

model.add(Flatten())
model.add(Dense(units=4096, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(units=4096, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(units=2, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


In [ ]:
import numpy as np

x_train = np.expand_dims(x_train, axis=-1)

x_train = np.repeat(x_train, 3, axis=-1)


In [ ]:

x_test = np.expand_dims(x_test, axis=-1)

x_test = np.repeat(x_test, 3, axis=-1)

In [ ]:
x_train.shape

In [ ]:
y_train

In [ ]:
import tensorflow as tf
tf.config.run_functions_eagerly(True)
model.fit(x_train, y_train, epochs=3, batch_size=32)


In [ ]:

test_loss, test_acc = model.evaluate(x_test, y_test)

print('Test accuracy:', test_acc)

In [ ]:
import matplotlib.pyplot as plt
import time

start_time = time.time()

history = model.fit(x_train, y_train, epochs=3)

end_time = time.time()

print("Total training time:", end_time - start_time, "seconds")


In [ ]:
start_time = time.time()

loss, test_accuracy = model.evaluate(x_test, y_test)

end_time = time.time()

print("Total testing time:", end_time - start_time, "seconds")


In [ ]:


plt.plot(history.history['accuracy'])
plt.plot(history.history['loss'])
plt.title('Accuracy vs. Loss')
plt.ylabel('Percentage')
plt.xlabel('Epoch')
plt.legend(['Accuracy', 'Validation'], loc='upper left')
plt.show()